In [ ]:
#import data
!pip install openpyxl
import pandas as pd
df = pd.read_excel(r"data\retail_pricing.xlsx")
df.head()

In [ ]:
#data cleaning&transformation
df = df.rename(columns={'time': 'date'})
df['date'] = (
    df['date']
    .astype(str)
    .str.strip()
    .pipe(pd.to_datetime, format='mixed')
)
df = df.sort_values('date')
df.head()

In [ ]:
#plotting rsi&cpi against date
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

fig, ax1 = plt.subplots(figsize=(10, 5))

# --- RSI (left axis) ---
line1, = ax1.plot(
    df['date'],
    df['retail_sales_index'],
    label='Retail Sales Index (RSI)'
)
ax1.set_ylabel('Retail Sales Index')

# --- CPI (right axis) ---
ax2 = ax1.twinx()
line2, = ax2.plot(
    df['date'],
    df['CPI'],
    linestyle='--',
    label='Consumer Price Index (CPI)'
)
ax2.set_ylabel('CPI')

# --- X-axis: show year only ---
ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# --- Shade post-inflation period ---
ax1.axvspan(
    pd.Timestamp('2021-01-01'),
    df['date'].max(),
    alpha=0.15
)

# --- Combined legend (KEY PART) ---
lines = [line1, line2]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

# --- Title & layout ---
plt.title('Singapore Retail Sales Index and CPI')
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
#transforming data into growth rate
df1 = df
df1['rsi_growth'] = df1['retail_sales_index'].pct_change(12) * 100
df1['cpi_growth'] = df1['CPI'].pct_change(12) * 100
df1 = df1.dropna()
df1

In [ ]:
#regression-1
import statsmodels.api as sm

X = sm.add_constant(df1['cpi_growth'])
y = df1['rsi_growth']

model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
#regression-2 (lagged model)
for lag in [0, 3, 6, 9]:
    df1[f'cpi_lag_{lag}'] = df1['cpi_growth'].shift(lag)

df1_lag = df1.dropna()

X = sm.add_constant(df1_lag[['cpi_lag_0', 'cpi_lag_3', 'cpi_lag_6']])
y = df1_lag['rsi_growth']

lag_model = sm.OLS(y, X).fit()
print(lag_model.summary())

In [ ]:
#plotting exchange rate
import pandas as pd
df2 = pd.read_excel(r"data\S$ Nominal Effective Exchange Rate Index.xlsx")
df2["date"].dtype

import matplotlib.pyplot as plt

# sort data by date
df = df.sort_values("date")

# define MAS tightening period(s)
# For example: May 2021 to May 2023 (adjust according to actual data)
tightening_start = pd.to_datetime("2021-12-01")
tightening_end   = pd.to_datetime("2025-09-01")

plt.figure(figsize=(12, 5))
plt.plot(df["date"], df["exchange_rate(usd/sgd)"], label="SGD Exchange Rate")

# highlight tightening period
plt.axvspan(tightening_start, tightening_end, color='orange', alpha=0.3, label="MAS Tightening Period")

plt.ylabel("Exchange Rate")
plt.title("SGD Exchange Rate with MAS Tightening Period Highlighted")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()
